# Opstellen database
### TO DO: PC6 data en PC4 data koppelen met PC6 code als index.
Data van statline is te vinden tot op buurtniveau, met hier en daar missende waarden. Hoe groter de schaal, hoe betrouwbaarder de data.
- Haal de data op buurtniveau op. Geef een totaal van de missende waarden.
- De data met adressen dient als relationele tabel. Zorg ervoor dat deze geschikt is.
- Met die tabel kan Statline-data gekoppeld worden aan PC6 data. Controleer of dit moet per huisnummer, straat of PC6 (eventueel PC4).
- Het moet mogelijk zijn om kolommen te kiezen met een tekstbestand. Minder kolommen is meer overzicht, zeker bij testen
- Meer informatie over de kolommen moet makkelijk te vinden zijn.
- Verpak alles in een overzichtelijke python file die geen specifieke namen van mappen aanneemt.
- Als meerdere uitslagen teglijk gebruikt worden is er multi-index of andere naamgeving van kolommen nodig. (Van toepassing op gebruik eerdere uitslagen als features)
- Data over criminaliteit moet apart van data.politie.nl gehaald worden.


In [2]:
# import database
code2020 = "84799NED"
code2019 = "84583NED" # Kerncijfers wijken en buurten 2019
code2017 = "83765NED" # Kerncijfers wijken en buurten 2017
# Toelichting: https://www.cbs.nl/nl-nl/maatwerk/2017/31/kerncijfers-wijken-en-buurten-2017
# en https://www.cbs.nl/nl-nl/maatwerk/2019/31/kerncijfers-wijken-en-buurten-2019

# Voor data die codes uit Statline koppelt aan postcodes, starten en huisnummers zie:
# https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer
postcodes = ("2019-cbs-pc6huisnr20190801_buurt/pc6hnr20190801_gwb.csv" , ";")
# De andere bestanden in die map bevatten de naam en code van respectievelijk buurten, wijken en gemeenten

# Kerncijfers wijken en buurten is niet in Odata4 beschikbaar.
# Downloaden data is het handigste, want Odata3 beperkt aantal rijen per verzoek.
# In omgeving van auteur zitten alle losse CSV bestanden in een map genaamd Data 
Statline2017 = "Kerncijfers_wijken_en_buurten_2017_22102020_140730.csv"
Statline2019 = "Kerncijfers_wijken_en_buurten_2019_22102020_140530.csv"
PC6_selectie = "CBS_PC6_selectie.csv"
EP2019 = "adressen_gl_prioriteit/adressen_EP2019.csv"
PS2019 = "adressen_gl_prioriteit/adressen_PS2019.csv"
TK2017 = "adressen_gl_prioriteit/adressen_TK2017.csv"
GR2018 = "adressen_gl_prioriteit/adressen_GR2018.csv"

Info over features te downloaden vanaf https://www.cbs.nl/nl-nl/maatwerk/2019/31/kerncijfers-wijken-en-buurten-2019
of als csv-bestand met

props = get_odata(url+"/DataProperties")
props.to_csv("MeasureCodes.csv", sep=";", na_rep="None")

Na het importeren van CBSparserOData3.get_odata (als deze af is) ...
Of download de CVS met features en de codes en wis de rijen met features die je niet wilt gebruiken. (Sla op als kopie)

In [3]:
import numpy as np
import pandas as pd
import CBSparserOData4
import re
import matplotlib.pyplot as plt
from scipy import stats

In [4]:
def laad_uitslagenPC6(path):
    uitslagen = pd.read_csv(path,sep="|")
    uitslagen['PC6'] = uitslagen['postcode']
    uitslagen['PC4'] = uitslagen['postcode'].str[0:4]
    uitslagen.drop(['postcode','huisnummertoevoeging','letter'], inplace = True, axis=1)
    register = uitslagen[['stad','straatnaam','stembureau','postcode_stembureau','PC6','PC4']]
    
    # Voor het gemak brengen we de data terug naar PC6 niveau, want de CBS data heeft toch niet meer nauwkeurigheid.
    # Later kan worden gecorrigeerd voor verschillen tussen PC6 en de zone rondom een stembureau.
    # Voordat de data gegroepeerd wordt moeten tabellen voor het koppelen van tekstkolommen bewaard worden.
    # Want een groupby wist alle tekstkolommen.
    # verkiezing = demo['verkiezing'].values[0] # Deze waarde is nodig mochten meerdere verkiezingen tegelijk geladen worden.
    
    uitslagen=uitslagen.groupby('PC6').median()
    #uitslagen=uitslagen.groupby(['postcode_stembureau','straatnaam']).median()
    #uitslagen=uitslagen.groupby(['PC6','straatnaam']).median()
    
    return uitslagen, register

In [5]:
def laad_CBSdataPC6(path):
    demo = pd.read_csv(path, sep=",", index_col=0)
    demo = demo.groupby('PC6').median()
    
    return demo
    

Er is ook een lijst nodig van álle postcodes voor een volledige dataset op PC6 niveau. Alle PC4 is het minimum. Het koppelen van de codes van bijbehorende wijk, buurt en gemeente is hierbij handig.

In [6]:
data_postcodes = pd.read_csv(postcodes[0], sep = postcodes[1])
data_postcodes = data_postcodes.groupby('PC6').median().drop('Huisnummer', axis=1)
for col in data_postcodes.columns:
    data_postcodes[col] = data_postcodes[col].astype('int')
print(data_postcodes.head())
print(data_postcodes.shape)

        Buurt2019  Wijk2019  Gemeente2019
PC6                                      
1011AB    3630400     36304           363
1011AC    3630400     36304           363
1011AD    3630400     36304           363
1011AE    3630400     36304           363
1011AG    3630403     36304           363
(460289, 3)


In [7]:
onderzoek = pd.read_csv('Data/CBS_PC6_selectie.csv', sep=",", index_col=0)
print(onderzoek.head(5))
print(onderzoek.shape)

      PC6  INWONER   MAN  VROUW  INW_014  INW_1524  INW_2544  INW_4564  \
1  1011AB      5.0   5.0    0.0      0.0       0.0       5.0       0.0   
2  1011AC     25.0  15.0   10.0      0.0       0.0      10.0       0.0   
3  1011AD      0.0   0.0    0.0      0.0       0.0       0.0       0.0   
4  1011AE      0.0   0.0    0.0      0.0       0.0       0.0       0.0   
5  1011AG     10.0   5.0    0.0      0.0       0.0       5.0       0.0   

   INW_65PL  AANTAL_HH  GEM_HH_GR  WONING  WON_MRGEZ  UITKMINAOW  \
1       0.0        5.0        1.4     0.0        0.0         0.0   
2       5.0       20.0        1.5     5.0        5.0         0.0   
3       0.0        0.0        NaN     0.0        0.0         0.0   
4       0.0        0.0        NaN     0.0        0.0         0.0   
5       0.0        5.0        1.6     0.0        0.0         0.0   

   Perc_NW_migracht  Gemcode2019 Gemeentenaam2019  
1               NaN          363        Amsterdam  
2              20.0          363        Am

In [8]:
onderzoek.describe()

,INWONER,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,AANTAL_HH,GEM_HH_GR,WONING,WON_MRGEZ,UITKMINAOW,Perc_NW_migracht,Gemcode2019
count,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,156810.000000,137324.000000,156810.000000,156810.000000,156810.000000,124604.000000,157023.000000
mean,40.477074,19.937217,20.351189,5.521236,4.539825,11.357566,9.906256,5.455232,20.023277,2.084678,18.889133,10.397519,2.923155,20.658807,415.164422
std,37.923528,19.741885,19.367376,8.640256,11.503882,15.545825,10.536750,10.964677,22.160447,0.620100,19.956008,20.454487,7.824927,23.333988,240.324323
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.000000
25%,20.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.600000,10.000000,0.000000,0.000000,0.000000,228.000000
50%,35.000000,15.000000,20.000000,0.000000,0.000000,10.000000,10.000000,0.000000,20.000000,2.000000,15.000000,0.000000,0.000000,10.000000,363.000000
75%,50.000000,25.000000,25.000000,10.000000,5.000000,15.000000,15.000000,10.000000,25.000000,2.500000,25.000000,20.000000,5.000000,30.000000,546.000000
max,2020.000000,1385.000000,1145.000000,290.000000,1365.000000,880.000000,580.000000,335.000000,2020.000000,6.000000,800.000000,800.000000,1290.000000,100.000000,983.000000


In [9]:
# Hoeveel data hebben we niet in de 'selectie'?
ontbrekend = set(data_postcodes.index) - set(onderzoek['PC6'])
len(ontbrekend)

303266

In [10]:
ontbrekend

{'5701BN',
 '7957CD',
 '3371HN',
 '2342CM',
 '9202KX',
 '5703XP',
 '7558LD',
 '5802ND',
 '6971LA',
 '8309AL',
 '6181GH',
 '6987DD',
 '6591WJ',
 '8647SC',
 '7451XD',
 '6131JV',
 '1741GL',
 '2352KD',
 '3362EE',
 '2396VB',
 '7651CP',
 '2924VE',
 '3224TG',
 '3772DB',
 '1962EH',
 '5133BC',
 '5261LW',
 '7271XA',
 '9601GS',
 '1689LM',
 '2861XC',
 '4006CH',
 '4337GA',
 '5708HX',
 '9883PR',
 '5524AW',
 '4759CG',
 '3204TP',
 '1871SK',
 '6634AR',
 '3417RJ',
 '2717AK',
 '4103WC',
 '1732EL',
 '2995VL',
 '8103EM',
 '2408JK',
 '7587RP',
 '1274CD',
 '3144PB',
 '3145RE',
 '5461DZ',
 '2721CJ',
 '5551ZD',
 '5401NH',
 '4681SW',
 '1444TC',
 '4311RS',
 '9461DC',
 '8312AN',
 '3765WN',
 '4254AT',
 '6137BN',
 '8225TH',
 '8131GS',
 '1945PN',
 '3898LA',
 '4871HL',
 '8448VE',
 '2241VK',
 '6164GP',
 '7596RA',
 '2406EV',
 '7274ER',
 '4462MD',
 '9163JB',
 '5066GP',
 '4691CR',
 '7607AN',
 '6021JT',
 '4341ED',
 '4462CJ',
 '4904AC',
 '2631EZ',
 '2245VZ',
 '6601GE',
 '8302VG',
 '2391NZ',
 '1774MK',
 '3999NT',
 '8071SX',

Normaliseer met aantal inwoners: MAN, VROUW, INW_xxx, UITKMINAOW,
Normaliseer met aantal huishoudens: WON_MRGEZ, 
Laat weg: INWONER, AANTAL_HH, Perc_NW_migracht. Deze laatse kolom bevat veel missende waarden.

In [11]:
def join_PC6(uitslagen,demo):
    # Omdat de data per PC6 is gemeten is het logisch om eerst met de PC6 de databases te koppelen.
    return uitslagen.join(demo)

In [12]:
feature_select = ['MAN','VROUW','UITKMINAOW','GEM_HH_GR']
# 'WON_MRGEZ' delen door aantal huishoudens levert niet een fractie van aantal huishoudens(waarde tussen 0 en 1) op.
# Gebruik deze feature pas als we weten wat het betekent.

In [13]:
def normaliseer_PC6(df, features=None, dropNA=True, drempel = None):
    # verwijder ook NaN's ?
    
    # Lijsten van alle mogelijke features, opgedeeld in hoe ze verwerkt moeten worden
    INW = [] # Wordt gevuld met inwoners in leeftijdsgroep en features in overig_persoon
    perc = ['stemperc'] # Opkomst en percentages stemmen worden altijd toegevoegd.
    huish = ['WON_MRGEZ']
    overig_persoon = ['MAN','VROUW','UITKMINAOW']
    overig_normaal = ['GEM_HH_GR'] # Features die al genormaliseerd zijn.
    zwarte_lijst = ['Perc_NW_migracht'] # Teveel missende waarden in deze kolom
    
    # verwijder kolommen 
    def overbodig(lijst, checklist):
        for c in lijst:
            if c not in checklist:
                lijst.remove(c)
        return lijst
    
    if features:
        huish = overbodig(huish, features)
        overig_persoon = overbodig(overig_persoon, features)
        overig_normaal = overbodig(overig_normaal, features)
        
    if drempel:
        stuk = df[df['INWONER'] >= drempel]
        df = df.loc[ stuk.index ]
        
    for col in df.columns:
        if col in overig_persoon or re.search('^INW_',col):
            df[col] = df[col] / df['INWONER']
            INW.append(col)
        elif col in huish:
            df[col] = df[col] / df['AANTAL_HH']
        elif col in zwarte_lijst:
            df.drop(col, axis=1,inplace=True)
        elif re.search('percentage$',col):
            perc.append(col)
            
    if features:
        cols = INW+overig_normaal+huish+perc
        print(cols)
        df = df[cols]
        
    if dropNA:
        df = df.dropna(axis=0)
    
    return df

In [38]:
EP2019, register = laad_uitslagenPC6("adressen_gl_prioriteit/adressen_EP2019.csv")
EP2019.head()

C:\Users\DKuipers\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (7,24) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,nummer,lat,lon,stemperc,prioriteit,totaal,opkomst,D66_stemmen,GL_stemmen,PvdA_stemmen,PvdD_stemmen,SP_stemmen,DENK_stemmen,D66_percentage,GL_percentage,PvdA_percentage,SP_percentage,DENK_percentage
PC6,,,,,,,,,,,,,,,,,,
1011AB,111.0,52.377773,4.905684,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.011990,0.019185
1011AC,147.0,52.377385,4.908563,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.011990,0.019185
1011AD,1.0,52.376820,4.904492,0.741339,0.0,433.0,0.481914,54.0,118.0,102.0,36.0,11.0,0.0,0.124711,0.272517,0.235566,0.025404,0.000000
1011AE,145.0,52.376399,4.910231,0.796163,0.0,417.0,0.429675,51.0,153.0,80.0,35.0,5.0,8.0,0.122302,0.366906,0.191847,0.011990,0.019185
1011AG,98.5,52.375865,4.902948,0.741339,0.0,433.0,0.481914,54.0,118.0,102.0,36.0,11.0,0.0,0.124711,0.272517,0.235566,0.025404,0.000000


In [39]:
demo = laad_CBSdataPC6('Data/'+PC6_selectie)
demo.head()

,INWONER,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,AANTAL_HH,GEM_HH_GR,WONING,WON_MRGEZ,UITKMINAOW,Perc_NW_migracht,Gemcode2019
PC6,,,,,,,,,,,,,,,
1011AB,5.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,1.4,0.0,0.0,0.0,NaN,363
1011AC,25.0,15.0,10.0,0.0,0.0,10.0,0.0,5.0,20.0,1.5,5.0,5.0,0.0,20.0,363
1011AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363
1011AE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363
1011AG,10.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,1.6,0.0,0.0,0.0,NaN,363


In [20]:
DataPC6 = join_PC6(EP2019,demo)
#print(DataPC6.columns)
print(DataPC6.head())
print(DataPC6.shape)

#test = DataPC6[DataPC6['INWONER'] >= 10]
#DataPC6.loc[test.index]

backup = DataPC6[['INWONER','prioriteit']]
DataPC6 = normaliseer_PC6(DataPC6,feature_select, drempel = 10)
DataPC6.describe()

        nummer        lat       lon  stemperc  prioriteit  totaal   opkomst  \
PC6                                                                           
1011AB   111.0  52.377773  4.905684  0.796163         0.0   417.0  0.429675   
1011AC   147.0  52.377385  4.908563  0.796163         0.0   417.0  0.429675   
1011AD     1.0  52.376820  4.904492  0.741339         0.0   433.0  0.481914   
1011AE   145.0  52.376399  4.910231  0.796163         0.0   417.0  0.429675   
1011AG    98.5  52.375865  4.902948  0.741339         0.0   433.0  0.481914   

        D66_stemmen  GL_stemmen  PvdA_stemmen  ...  INW_2544  INW_4564  \
PC6                                            ...                       
1011AB         51.0       153.0          80.0  ...       5.0       0.0   
1011AC         51.0       153.0          80.0  ...      10.0       0.0   
1011AD         54.0       118.0         102.0  ...       0.0       0.0   
1011AE         51.0       153.0          80.0  ...       0.0       0.0   
10

,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,UITKMINAOW,GEM_HH_GR,stemperc,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
count,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000,129815.000000
mean,0.490464,0.502954,0.113560,0.094380,0.269591,0.240802,0.137576,0.066936,2.077614,0.566533,0.087225,0.160242,0.197828,0.052725,0.038969,0.029522
std,0.110360,0.110448,0.120404,0.133112,0.183897,0.148877,0.193655,0.113435,0.617161,0.117324,0.035926,0.075624,0.042880,0.019384,0.020908,0.058111
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.049505,0.005780,0.000000,0.014851,0.000000,0.000000,0.000000
25%,0.428571,0.444444,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,1.600000,0.486239,0.059181,0.100592,0.169001,0.039062,0.023307,0.001855
50%,0.500000,0.500000,0.100000,0.000000,0.266667,0.250000,0.066667,0.000000,2.000000,0.561333,0.081744,0.142241,0.195041,0.051339,0.035398,0.008163
75%,0.555556,0.571429,0.200000,0.157895,0.384615,0.333333,0.200000,0.117647,2.500000,0.656842,0.112219,0.212316,0.223479,0.064572,0.050866,0.029412
max,1.000000,1.000000,0.800000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,0.907937,0.217445,0.438486,0.363853,0.184035,0.139130,0.585034


In [21]:
def prefixer(s, prefix, lengte):
    s = str(s)
    while len(s) < lengte:
        s = '0' + s
    s = prefix + s
    return s

def get_index():
    index = index.groupby('PC6').median()
    index['Buurt2019'] = index['Buurt2019'].astype('int').apply(lambda s: prefixer(s,'BU', 8))
    index['Wijk2019'] = index['Wijk2019'].astype('int').apply(lambda s: prefixer(s, 'WK', 6))
    index['Gemeente2019'] = index['Gemeente2019'].astype('int').apply(lambda s: prefixer(s, 'GM', 4))
    return index 

def get_prioriteit(df):
    if 'prioriteit' in df.columns:
        if 'PC6' in df.columns:
            return(df[['PC6','prioriteit']])
        else:
            return df.prioriteit

## Koppelen Data Statline

In [18]:
data_2019 = pd.read_csv('Data/'+Statline2019, sep=';', decimal=b',')
data_2019.head()

,Wijken en buurten,Regioaanduiding/Soort regio (omschrijving),Bevolking/Geboorte en sterfte/Geboorte totaal (aantal),Bevolking/Geboorte en sterfte/Geboorte relatief (per 1 000 inwoners),Bevolking/Geboorte en sterfte/Sterfte totaal (aantal),Bevolking/Geboorte en sterfte/Sterfte relatief (per 1 000 inwoners),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Appartement (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Tussenwoning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Hoekwoning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Twee-onder-één-kap-woning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Vrijstaande woning (kWh),Nabijheid voorzieningen/Basisonderwijs/Afstand tot school (km),Nabijheid voorzieningen/Basisonderwijs/Scholen binnen 3 km (aantal)
0,Altena,Gemeente,563,10,444,8,2050.0,2780.0,2930.0,3450.0,4150.0,0.7,2.7
1,Werkendam,Wijk,130,12,100,9,2070.0,2800.0,2890.0,3500.0,4380.0,0.8,3.9
2,Centrum Werkendam,Buurt,20,11,10,5,2290.0,2730.0,2700.0,3470.0,3960.0,1.0,4.0
3,Burchtpolder,Buurt,20,14,15,10,1990.0,2400.0,2820.0,3440.0,3950.0,0.6,4.0
4,Beekdaelen,Gemeente,263,7,381,11,2170.0,2990.0,3100.0,3210.0,4010.0,0.9,3.3


In [19]:
# Kan ook prima de csv aanmaken zonder deze functie.
def laad_Statline(string, sep=';', downloaded=True, version=3, zoekterm='Wijk'):
    """Laad data afkomstig van Statline standaard (Odata3).
    Data die al is gedownload als CSV moet extra gecontroleerd worden.
    Standaard worden alleen de wijken behouden, vul anders de zoekterm in."""
    
    if downloaded:
        df = pd.read_csv(string, sep=sep)
        df = df[df["Regioaanduiding/Soort regio (omschrijving)"].str.strip() == zoekterm].drop("Regioaanduiding/Soort regio (omschrijving)",axis=1)
    elif version == 3:
        import CBSparserOData3
        df = CBSparserOData3.get_observations(string,filtersoort = 2, plaatsen = zoekterm)
    elif version == 4:
        import CBSparserOData4
        if wijken == 'Wijk':
            print("Foutmelding: In OData4 moet de zoekterm 'BU, WK, of GM zijn.'")
            pass
        df = CBSparserOData4.parserOData4(string,zoekterm)
    else:
        pass
    
    # Beperk aantal kolommen?
    for col in df:
            if df[col].dtype == np.dtype('O'):
                if re.match("\d,\d", df[col].values[0]):
                    df[col] = df[col].apply(lambda s: re.sub(',','.',s))
                    df[col] = df[col].astype('float32')
    return df

In [22]:
#test = laad_Statline("84799NED", downloaded=False, version=3, zoekterm="Wijk")
#test.head()

In [24]:
test = laad_Statline("Data/Kerncijfers_wijken_en_buurten_2019_22102020_140530.csv")
test.head()

,Wijken en buurten,Bevolking/Geboorte en sterfte/Geboorte totaal (aantal),Bevolking/Geboorte en sterfte/Geboorte relatief (per 1 000 inwoners),Bevolking/Geboorte en sterfte/Sterfte totaal (aantal),Bevolking/Geboorte en sterfte/Sterfte relatief (per 1 000 inwoners),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Appartement (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Tussenwoning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Hoekwoning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Twee-onder-één-kap-woning (kWh),Energie/Gemiddeld elektriciteitsverbruik/Naar woningtype/Vrijstaande woning (kWh),Nabijheid voorzieningen/Basisonderwijs/Afstand tot school (km),Nabijheid voorzieningen/Basisonderwijs/Scholen binnen 3 km (aantal)
1,Werkendam,130,12,100,9,2070.0,2800.0,2890.0,3500.0,4380.0,0.8,3.9
5,Wijk 00 Onderbanken,65,8,80,11,2050.0,3120.0,3170.0,3230.0,4100.0,0.7,2.7
9,Centrum,80,4,90,4,1930.0,2500.0,2990.0,2590.0,3150.0,0.6,16.4
13,Hoofddorp,720,10,485,6,1940.0,3050.0,3180.0,3990.0,4520.0,0.7,14.4
17,Oud-Beijerland,215,9,220,9,2050.0,2960.0,3160.0,3490.0,4170.0,0.8,8.1


In [57]:
def maak_database(postcodes, uitslagen, PC6, Statline=None, prioriteit=None, features=None, features_Statline=None, save_as=None):
    """Combineer stemuitslagen met data van het CBS, eventueel met data van Statline om lege plekken op te vullen.
    Het opvullen van missende data met geaggregreede cijfers is een goede benadering maar niet even betrouwbaar.
    Geef twee of drie pandas dataframes op. De eerste moet afkomstig zijn van https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer
    Uitslagen en PC6 mogen ook een 'path' zijn. De eerste drie datasets moeten PC6 als index hebben.
    !!! De naamgeving van de kolommen in de PC6 data en OData3 is niet hetzelfde. Dit moet eerst worden afgestemd of handmatig worden gecorrigeerd in deze functie.
    PC6 die nog ontbreken worden geaggregeerd tot PC4 en daar wordt de Statline Data bij gezocht.
    Als features een lijst is worden uit ALLE TABELLEN alleen kolommen gebruikt als ze daar in voorkomen.
    Zorg ervoor dat je weet hoe elke individuele tabel eruit ziet voordat je ze samenvoegd.
    Uitslagen mogen ook meerdere datasets zijn, er wordt dan gebruik gemaakt van prefixes.
    Functie kan data meteen opslaan en geeft altijd een paar cijfers over hoeveel postcodes er niet voorkomen en hoeveel er zij geaggregeerd.
    """
    
    if type(postcodes) == str:
        df_postcodes = pd.read_csv(postcodes, sep = ";")
        df_postcodes = df_postcodes.groupby('PC6').median().drop('Huisnummer', axis=1)
        #print(df_postcodes)
        
        for col in df_postcodes.columns:
            df_postcodes[col] = df_postcodes[col].astype('int')
    else:
        df_postcodes = postcodes
    
    # if list
    if type(uitslagen) == str:
        df_uitslagen = laad_uitslagenPC6(uitslagen)
    else:
        df_uitslagen = uitslagen

    if type(PC6) == str:
        df_PC6 = pd.read_csv(PC6, sep=",", index_col=0)
        df_PC6 = df_PC6.groupby('PC6').median()
        df_PC6 = normaliseer_PC6(df_PC6, features=features) # test features
    else:
        df_PC6 = PC6
    
    print(df_PC6.head())
    df_PC6 = df_PC6.join((df_uitslagen, df_postcodes), how='left')
    
    # if features
    # if prioriteit
    
    if Statline:
        df_Statline = laad_Statline(Statline)
        # join
    # if features_Statline
    
    if save_as:
        df_uitslagen.to_csv(save_as, sep=';')
    
    return df_PC6

In [58]:
test = maak_database(postcodes[0], EP2019, "Data/CBS_PC6_selectie.csv")
print(test.columns)
print(test.shape)
#EP2019.join((demo, data_postcodes), how='left')

        INWONER       MAN     VROUW  INW_014  INW_1524  INW_2544  INW_4564  \
PC6                                                                          
1011AB      5.0  1.000000  0.000000      0.0       0.0  1.000000  0.000000   
1011AC     25.0  0.600000  0.400000      0.0       0.0  0.400000  0.000000   
1011AG     10.0  0.500000  0.000000      0.0       0.0  0.500000  0.000000   
1011AH     25.0  0.600000  0.400000      0.0       0.0  0.800000  0.000000   
1011AJ     15.0  0.666667  0.333333      0.0       0.0  0.333333  0.333333   

        INW_65PL  AANTAL_HH  GEM_HH_GR  WONING  WON_MRGEZ  UITKMINAOW  \
PC6                                                                     
1011AB       0.0        5.0        1.4     0.0       0.00         0.0   
1011AC       0.2       20.0        1.5     5.0       0.25         0.0   
1011AG       0.0        5.0        1.6     0.0       0.00         0.0   
1011AH       0.0       15.0        1.7    15.0       1.00         0.0   
1011AJ       0.

## Controle kwaliteit data
In kolommen met een fractie (tussen 0 en 1) zitten nog opvallend veel datapunten met (bijna) precies 0 en 1.
Dat is erg vreemd, zeker gezien het feit dat eerder de mediaan per PC6 is genomen. (Gemiddelden hebben geen betekenis omdat het gaat om dezelfde meting met steeds een ander adres)
In welke kolommen speelt dit probleem, en in welke mate? Wat kunnen we zeggen over deze datapunten? Is dat genoeg reden om deze weg te laten?
Zie ook ???


### Uitkomst:
'GEM_HH_GR' heeft als minimum 1 en maximum 6, en is dus betrouwbaar.


De meest realistische mogelijkheid is dat we een minimum hanteren voor het aantal inwoners of huishoudens dat een PC6 moet hebben om gebruikt te mogen worden. De grootte van één straat is daarbij een goede vuistregel.

In [116]:
DataPC6 = DataPC6.join(backup['INWONER'])
for col in DataPC6.columns[0:9]:
    MIN = min(DataPC6[col])
    mins = DataPC6[ DataPC6[col] == MIN ][col]
    MAX = max(DataPC6[col])
    maxs = DataPC6[ DataPC6[col] == MAX ][col]
    print("In {} komt {}, het minimum, {} maal voor. {}, het maximum, komt {} maal voor.".format(
        col,MIN,len(mins),MAX,len(maxs)))

In MAN komt 0.0, het minimum, 1510 maal voor. 1.0, het maximum, komt 319 maal voor.
In VROUW komt 0.0, het minimum, 1363 maal voor. 1.0, het maximum, komt 394 maal voor.
In INW_014 komt 0.0, het minimum, 58422 maal voor. 0.8, het maximum, komt 1 maal voor.
In INW_1524 komt 0.0, het minimum, 67047 maal voor. 1.0, het maximum, komt 121 maal voor.
In INW_2544 komt 0.0, het minimum, 22906 maal voor. 1.0, het maximum, komt 240 maal voor.
In INW_4564 komt 0.0, het minimum, 21298 maal voor. 1.0, het maximum, komt 56 maal voor.
In INW_65PL komt 0.0, het minimum, 62850 maal voor. 1.0, het maximum, komt 975 maal voor.
In UITKMINAOW komt 0.0, het minimum, 85531 maal voor. 1.0, het maximum, komt 12 maal voor.
In GEM_HH_GR komt 1.0, het minimum, 1463 maal voor. 6.0, het maximum, komt 1 maal voor.


In [101]:
C = 'VROUW'
MIN = min(DataPC6[C])
#mins = DataPC6[ DataPC6[C] <= MIN ]
MAX = max(DataPC6[C])
#maxs = DataPC6[ DataPC6[C] >= MAX ]
extreem1 = DataPC6[DataPC6[C] >= MAX][[C,'INWONER']]
extreem2 = DataPC6[DataPC6[C] <= MIN][[C,'INWONER']]
print(extreem1.describe(),'\n',extreem2.describe())

       VROUW     INWONER
count  465.0  465.000000
mean     1.0   11.204301
std      0.0    5.338860
min      1.0    5.000000
25%      1.0   10.000000
50%      1.0   10.000000
75%      1.0   10.000000
max      1.0   45.000000 
         VROUW      INWONER
count  1677.0  1677.000000
mean      0.0    10.822898
std       0.0     5.142040
min       0.0     5.000000
25%       0.0    10.000000
50%       0.0    10.000000
75%       0.0    10.000000
max       0.0    95.000000


Kolom MAN: meer dan driekwart van minimum waarden heeft 10 inwoners of minder, meer dan driekwart van maximum waarden heeft 15 inwoners of minder.
Kolom VROUW: meer dan driekwart van minimum waarden heeft 10 inwoners of minder, meer dan driekwart van maximum waarden heeft 15 inwoners of minder.

In [ ]:
# Met dank aan https://www.w3schools.com/python/python_ml_linear_regression.asp
plot_selectie = ['MAN','VROUW']
y_feature = 'GL_percentage'
fig, ax = plt.subplots( len(plot_selectie) )
kolommen = ['slope','intercept','r','p','std_err']
rijen = []

for n, feature in enumerate(plot_selectie):
    x = DataPC6[feature]
    slope, intercept, r, p, std_err = stats.linregress(x, DataPC6[y_feature])
    rij = pd.Series((slope,intercept,r,p,std_err), index = kolommen)
    rijen.append(rij)
    
    def linfunc(x):
        return slope*x + intercept
    linmodel = list(map(linfunc, x))
    ax[n].scatter(x,dataPC6[y_feature], s=0.1)

data = pd.DataFrame(rijen, index = plot_selectie)
plt.show()

Slimmer indelen:
Maak een functie die in de oorspronkelijke tabel informatie uit tekstkolommen ophaalt na het invoeren van een PC6 (van stembureau). Laat deze een kolom aanmaken die aangeeft of er straten (of adressen) zijn binnen dat kader die buiten die PC6 vallen. Bijvoorbeeld met een cijfer dat het aantal combinaties aangeeft.